Original code from https://github.com/primaryobjects/oracle, modified by Hanadi Mohamed. 

In [1]:
pip install tweedledum


[notice] A new release of pip available: 23.1.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

util

In [3]:
import ast
import random
import string
import time
import pathlib
import configparser
from math import ceil, log
from configparser import RawConfigParser
from qiskit import qiskit, Aer, IBMQ, QuantumCircuit

def random_letters(n):
    # Generate an array of random letters.
    letters = []
    for i in range(n):
        letters.append(random.choice(string.ascii_letters))
    return letters

def random_number(minimum, maximum):
    # Uses a quantum circuit to generate a random number from minimum (inclusive) to maximum (exclusive).
    # Determine the number of qubits required.
    n = num_qubits(maximum-1)

    # Create a quantum circuit with enough qubits for the max value.
    qc = QuantumCircuit(n)

    # Place all qubits into superposition.
    qc.h(range(n))

    # Measure the result.
    qc.measure_all()

    # Continue executing the circuit until we obtain a value within range.
    r = -1
    count = 0
    max_count = 10
    while (r < minimum or r >= maximum) and count < max_count:
        # Execute the circuit.
        result = execute(qc)

        # Get the resulting hit counts.
        counts = result.get_counts()

        # Find the most frequent hit count.
        key = max(counts, key=counts.get)

        # Since the quantum computer returns a binary string (one bit for each qubit), we need to convert it to an integer.
        r = int(key, 2)

        # Increment the count before we break.
        count = count + 1

    return r

def num_qubits(i):
    # Returns the number of qubits needed to represent the value i.
    return ceil(log(i, 2))

def execute(qc):
    # Setup the API key for the real quantum computer.
   # parser = RawConfigParser()
   # parser.read('config.ini')

    # Read configuration values.
   # proxies = ast.literal_eval(parser.get('IBM', 'proxies')) if parser.has_option('IBM', 'proxies') else None
   # verify = (True if parser.get('IBM', 'verify') == 'True' else False) if parser.has_option('IBM', 'verify') else True
    #token = parser.get('IBM', 'key')
    is_sim = True
   # is_sim = token == 'YOUR_API_KEY'
    #is_sim = token == 'tsAI7_sxy4XIIRfscBAIKPRJ1pzfD6yDqxw8phZ1b9vO'
    
   # if not is_sim:
   #     execute.provider = execute.provider or IBMQ.enable_account(token = token, proxies = proxies, verify = verify) or IBMQ.load_account()
    #    backend = execute.provider.backends(simulator=False)[1]
   # else:
    backend = Aer.get_backend('aer_simulator')

    if not is_sim:
        print("Running on", backend.name())

    start = time.time()
    job = qiskit.execute(qc, backend)
    result = job.result()
    stop = time.time()

    if not is_sim:
        print("Request completed in " + str(round((stop - start) / 60, 2)) + "m " + str(round((stop - start) % 60, 2)) + "s")

    return result

grover

In [4]:
from qiskit import QuantumCircuit
from qiskit.circuit.classicalregister import ClassicalRegister
from qiskit.circuit.quantumregister import QuantumRegister

def diffuser(n):
    qc = QuantumCircuit(n)

    # Apply transformation |s> -> |00..0> (H-gates)
    qc.h(range(n))

    # Apply transformation |00..0> -> |11..1> (X-gates)
    qc.x(range(n))

    # Do multi-controlled-Z gate
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1)  # multi-controlled-toffoli
    qc.h(n-1)

    # Apply transformation |11..1> -> |00..0>
    for qubit in range(n):
        qc.x(qubit)

    # Apply transformation |00..0> -> |s>
    qc.h(range(n))

    # We will return the diffuser as a gate
    gate = qc.to_gate()
    gate.name = "diffuser"

    return gate

def grover(oracle, logic, n):
    # The circuit is a Grover's search for the all-ones state.
    var = QuantumRegister(n, 'var')
    out = QuantumRegister(1, 'out')
    cr = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(var, out, cr)

    # Initialize the output qubit to a phase-flip.
    qc.x(n)
    qc.h(n)

    # Apply the Hadamard gate to every qubit.
    qc.h(var)
    qc.barrier()

    # Apply the oracle to every qubit.
    qc.append(oracle(logic, n), range(n+1))
    qc.barrier()

    # Apply the diffuser to every qubit.
    qc.append(diffuser(n), range(n))
    qc.barrier()

    # Undo the output qubit phase-flip.
    qc.h(n)
    qc.x(n)

    qc.measure(var, cr)
    #Testing
    #qc.draw("mpl")
    return qc

logic

In [5]:
from qiskit import QuantumCircuit
from qiskit.circuit.classicalfunction.classicalfunction import ClassicalFunction

def oracle(logic, n):
    '''
    Returns a quantum circuit that implementes the logic for n qubits.
    Parameters:
    logic: a Python function using the format below.
            def oracle_func(x1: Int1, x2: Int1, x3: Int1) -> Int1:\n  return (x1 and not x2 and not x3)
    n: the number of qubits in the circuit.
    '''

    # Convert the logic to a quantum circuit.
    formula = ClassicalFunction(logic)
    fc = formula.synth()

    # Convert the quantum circuit to a quantum program.
    qc = QuantumCircuit(n+1)
    qc.compose(fc, inplace=True)

    print(qc.draw())

    # Convert the oracle to a gate.
    gate = qc.to_gate()
    gate.name = "oracle"

    return gate

In [6]:
import random
from math import floor, log

def init(target, n):
    # Initialize an array of random letters, including the target ones in the string.
    arr = random_letters(n)

    # Store used indices so that we don't overwrite target letters.
    indices = [-1]

    # Select random indices for the target letters.
    for i in range(len(target)):
        index = -1

        # Select a random index until we find one that hasn't been used.
        while index in indices:
            index = random.randint(0, n - 1)

        # Set the target letter at the selected index.
        arr[index] = target[i]

        # Add the index to the list of used indices.
        indices.append(index)
    return arr

def logic(arr, target, n):
    # Finds the indices within arr for each position of target letter and returns a Python function as a string to be used in an oracle.
    prog = '''def oracle_func(_PARAMS_) -> Int1:
            return _CLAUSE_'''

    # Insert the parameters for the function, based upon the number of qubits needed for target.
    qubits = floor(log(n, 2))
    params = ''
    for i in range(qubits):
        # x1: Int1, x2: Int1, x3: Int1
        params += 'x' + str(i + 1) + ': Int1, '

    # Remove the trailing comma.
    params = params[:-2]
    prog = prog.replace('_PARAMS_', params)

    # Insert the clauses for the function, based upon the bit location(s) for the target letter.
    # (not x1 and not x2 and not x3) or (x1 and x2 and not x3)
    clauses = ''
    # Find the next index of the target letter.
    index = arr.index(target, 0)
    while index != -1:
        # Convert the index to a binary string.
        bin_str = bin(index)[2:]

        # Pad the binary string with zeros to the length of the qubits.
        bin_str = bin_str.zfill(qubits)

        # Generate the logical string for the clause.
        clause = ''
        for j in range(len(bin_str)-1, -1, -1):
            if bin_str[j] == '0':
                clause += 'not '
            clause += 'x' + str(len(bin_str) - j) + ' and '

        # Trim trailing 'and ' from the clause.
        clause = clause[:-5]

        # Add parentheses to the clause.
        clause = '(' + clause + ')'

        # Insert the clause into the clauses.
        clauses += clause + ' or '

        # Find the next index matching the letter (if any more exist).
        try:
            index = arr.index(target, index + 1)
        except ValueError:
            # No more letters found.
            index = -1

    # Trim trailing 'or ' from the clauses.
    clauses = clauses[:-4]
    prog = prog.replace('_CLAUSE_', clauses)

    return prog



In [7]:
def main(phrase):
    execute.provider = None

    # Calculate the number of qubits needed to represent the number of letters in the target.
    qubits = num_qubits(len(phrase))

    bits = 2 ** qubits
    arr = init(phrase, bits)

    print(str(qubits) + ' qubits, ' + str(bits) + ' possibilities')
    print('Using random letters:')
    print(arr)

    # Use the oracle to find the indices of the target letters.
    indices = []
    for letter in phrase:
        print("Finding letter '" + letter + "'")

        # Generate a logical function for the oracle.
        prog = logic(arr, letter, bits)

        # Generate the quantum circuit.
        qc = grover(oracle, prog, qubits)
        print(qc)
        #qc.draw("mpl")
        # Execute the quantum circuit.
        result = execute(qc)

        # Get the resulting hit counts.
        counts = result.get_counts()
        print(counts)

        # Find the most frequent hit count.
        key = max(counts, key=counts.get)

        # Since the quantum computer returns a binary string (one bit for each qubit), we need to convert it to an integer.
        index = int(key, 2)

        # Print the resulting letter selected by the quantum circuit.
        print(arr[index])

        indices.append({'binary': key, 'index': index, 'letter': letter})

    print('\nRandom letters:')
    print(arr)
    print()

    print('Final result from the quantum circuit:\n')
    for i in range(len(indices)):
        letter = arr[indices[i]['index']]
        index = indices[i]['index']
        binary = indices[i]['binary']

        print(letter + ' (at index ' + str(index) + ' [' + binary + '])')

main('hello')

3 qubits, 8 possibilities
Using random letters:
['o', 'h', 'P', 'l', 'C', 'e', 'l', 'l']
Finding letter 'h'
Figure(203.885x367.889)
       ┌───┐      ░ ┌─────────┐ ░ ┌───────────┐ ░      ┌─┐           
var_0: ┤ H ├──────░─┤0        ├─░─┤0          ├─░──────┤M├───────────
       ├───┤      ░ │         │ ░ │           │ ░      └╥┘┌─┐        
var_1: ┤ H ├──────░─┤1        ├─░─┤1 diffuser ├─░───────╫─┤M├────────
       ├───┤      ░ │  oracle │ ░ │           │ ░       ║ └╥┘┌─┐     
var_2: ┤ H ├──────░─┤2        ├─░─┤2          ├─░───────╫──╫─┤M├─────
       ├───┤┌───┐ ░ │         │ ░ └───────────┘ ░ ┌───┐ ║  ║ └╥┘┌───┐
  out: ┤ X ├┤ H ├─░─┤3        ├─░───────────────░─┤ H ├─╫──╫──╫─┤ X ├
       └───┘└───┘ ░ └─────────┘ ░               ░ └───┘ ║  ║  ║ └───┘
  c: 3/═════════════════════════════════════════════════╩══╩══╩══════
                                                        0  1  2      
{'110': 28, '101': 43, '100': 33, '000': 35, '010': 26, '111': 34, '011': 29, '001': 796}
h
Findin